In [134]:
import pandas as pd
import numpy as np
import csv
import re

from unidecode import unidecode #para los acentos

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [146]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [147]:
df1 = pd.read_csv("../scraping/scrape_mercadona/mercadona-01.csv")
df2 = pd.read_csv("../scraping/scrape_mercadona/mercadona-02.csv")
df3 = pd.read_csv("../scraping/scrape_mercadona/mercadona-03.csv")
df4 = pd.read_csv("../scraping/scrape_mercadona/mercadona-04.csv")
df5 = pd.read_csv("../scraping/scrape_mercadona/mercadona-05.csv")
df6 = pd.read_csv("../scraping/scrape_mercadona/mercadona-06.csv")
df7 = pd.read_csv("../scraping/scrape_mercadona/mercadona-1.2.csv")

In [148]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

In [149]:
df.sample()

,categoria,subcategoria,descripcion,unidades,precio,code,compare
1144,Congelados,Rebozados,Merluza empanada pan fino Hacendado ultraconge...,Paquete 510 g,"4,20 €3,95 € /ud.",63140.0,"7,746 €/kg"


In [150]:
df.shape

(9282, 7)

## DUPLICADOS

In [151]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 4893 duplicate rows


In [152]:
df.drop_duplicates(inplace=True)

In [153]:
df.shape

(4389, 7)

## PRECIO

In [154]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        info: pos[0]: str 
        valores: pos[1]: float
        descuento: pos[2]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").split("/")
    
    # 2. Split by the "/" to extract the info, keep last part(always at the end)
    info = x[-1].replace(".", "") #(str)
    
    # 3. Splits by the "/" to extract the num values:
    num_values = x[0:-1][0].replace("," , ".").strip().split(" ")
    
    # 4. Extracts and formats price value:
    price = float(num_values[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(num_values) == 1 else float(num_values[1])][0]
    
    return [info, price, discount]

In [155]:
# Apply the function and get the element for each colum: 

"He decidodo que esto no aporta nada"
#df["price_info"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[2])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

In [156]:
df

,categoria,subcategoria,descripcion,unidades,code,compare,price_eu,price_disc_eu
0,"Aceite, especias y salsas","Aceite, vinagre y sal","Aceite de oliva 0,4º Hacendado",Botella 1 L,4240.0,"9,25 €/L",9.25,NaN
1,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Garrafa 3 L,4717.0,"9,85 €/L",29.55,NaN
2,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado,Botella 1 L,4740.0,"9,90 €/L",9.90,NaN
3,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva virgen extra Hacendado Gran Se...,Botella 750 ml,4706.0,"10,80 €/L",8.10,NaN
4,"Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva 1º Hacendado,Botella 1 L,4640.0,"9,25 €/L",9.25,NaN
...,...,...,...,...,...,...,...,...
1847,Zumos,Tomate y otros sabores,Néctar de maracuyá Hacendado fruta de la pasió...,Brick 1 L,39643.0,"0,81 €/L",0.81,NaN
1848,Zumos,Tomate y otros sabores,Bebida de mango Hacendado sin azúcares añadidos,Brick 1 L,39132.0,"1,25 €/L",1.25,NaN
1849,Zumos,Tomate y otros sabores,Bebida de pomelo Hacendado sin azúcares añadidos,Brick 1 L,39604.0,"1,25 €/L",1.25,NaN
1850,Zumos,Tomate y otros sabores,Zumo de manzana selección Hacendado,Brick 1 L,39922.0,"1,50 €/L",1.50,NaN


## - MARCA

Hay tres marcas principales que se van a repetir en casi la mitad de articulos, que son los nombres
de las marcas propias de mercadona: 


In [157]:
"""Funcion que te devuelve la marca si el texto la contiene"""
def marca(x):
    #marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola"]
    
    marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola", "Aquarius", "Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares", "Mahou"]
    
    
    return [i for i in marcas if i in x]

In [158]:
#Aplicamos la funcion, como realmente devuelve una lista, si esta vacia (no match) --> np.nan
df["marca"] = df["descripcion"].apply(marca).apply(lambda x: x[0] if x != [] else np.nan)

In [159]:
#Quiero ver cuantas filas no hemos identificado la marca 
df['marca'].isna().sum()

1520

In [160]:
df.sample()

,categoria,subcategoria,descripcion,unidades,code,compare,price_eu,price_disc_eu,marca
568,Cereales y galletas,Cereales,Avena molida Hacendado,Paquete 500 g,16807.0,"2,22 €/kg",1.11,NaN,Hacendado


### UNIDADES:

In [161]:
# Por el tema de acentos
df["unidades"] = df["unidades"].apply(unidecode)

#con esto eliminos los puntos de ud. y de aprox.
df["unidades"] = df["unidades"].str.replace(".", "")

# me quito todods los aprox
df["unidades"] = df["unidades"].str.replace("aprox", "")

df["unidades"] = df["unidades"].str.replace("(", "")

df["unidades"] = df["unidades"].str.replace(")", "")

In [162]:
def format_units(i):
    
    # Format Ex:         (contains "escurrido")
    pattern_1 = r'^(\S+)\s+(\d+(?:,\d+)?\s*[kKgGmlML]+)\s+(\d+\s*[kKgGmlML]+)\s*escurrido$'

    # Fromat Ex:
    pattern_2 = r'^([A-Z][a-zA-Z]+)\s((?:\d+,)?\d+\s(?:[Kk]g|g|L|ml))'

    #Formaat Ex: '8 latas x 330 ml
    pattern_3 = r'(\d+)\s([a-zA-Z\s]+)\sx\s((?:\d+,)?\d+\s(?:kg|ml|g|L))'
    
    #Format Ex:
    pattern_4 = r'(?:\d+,)?\d+\s+(?:kg|g|ml|L)'
    
    
    if re.match(pattern_1, i):

        matches = re.findall(pattern_1, i)

        formato = matches[0][0]  #.capitalize() ya veremos 

        cantidad = f'1 x {matches[0][2]}'

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]


    # Formato 1: Paquete 500 g / Tarro 570 g (400 g escurrido)
    if re.match(pattern_2, i):

        matches = re.findall(pattern_2, i)

        formato = matches[0][0]  #.capitalize() ya veremos 

        cantidad = f'1 x {matches[0][1]}'

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]


    #Formato 2: '8 latas x 330 ml
    if re.match(pattern_3, i):

        matches = re.findall(pattern_3, i)

        cantidad = f'{matches[0][0]} x {matches[0][2]}'
        formato = matches[0][1]  #.capitalize() ya veremos 

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]

    if re.search(pattern_4, i):

        matches = re.findall(pattern_4, i)

        cantidad = f'1 x {matches[0]}'

        formato = i.replace(matches[0], "")

        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]

    else:
        formato = i
        cantidad = np.nan
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]




In [163]:
df["format"] = df["unidades"].apply(format_units).apply(lambda x:x[0])
df["quantity"] = df["unidades"].apply(format_units).apply(lambda x:x[1])

In [164]:
# POR AHORA LO VOY A DEJAR COMO ESTA -- LAS UNIDADES DIGO:

#Drop old column:
#df.drop("unidades", axis=1, inplace=True)

In [165]:
df.rename(columns={"categoria":"category", 
                   "subcategoria":"subcategory", 
                   "descripcion":"description",
                    "unidades":"units",
                    "code": "product_code",
                      "marca" : "brand"}, inplace = True)

In [166]:
categories = df[["product_code","description", "category", "subcategory"]]

In [167]:
export_to_csv(categories, "codes-categories")

DataFrame successfully exported to codes-categories.csv


In [168]:
categories

,product_code,description,category,subcategory
0,4240.0,"Aceite de oliva 0,4º Hacendado","Aceite, especias y salsas","Aceite, vinagre y sal"
1,4717.0,Aceite de oliva virgen extra Hacendado,"Aceite, especias y salsas","Aceite, vinagre y sal"
2,4740.0,Aceite de oliva virgen extra Hacendado,"Aceite, especias y salsas","Aceite, vinagre y sal"
3,4706.0,Aceite de oliva virgen extra Hacendado Gran Se...,"Aceite, especias y salsas","Aceite, vinagre y sal"
4,4640.0,Aceite de oliva 1º Hacendado,"Aceite, especias y salsas","Aceite, vinagre y sal"
...,...,...,...,...
1847,39643.0,Néctar de maracuyá Hacendado fruta de la pasió...,Zumos,Tomate y otros sabores
1848,39132.0,Bebida de mango Hacendado sin azúcares añadidos,Zumos,Tomate y otros sabores
1849,39604.0,Bebida de pomelo Hacendado sin azúcares añadidos,Zumos,Tomate y otros sabores
1850,39922.0,Zumo de manzana selección Hacendado,Zumos,Tomate y otros sabores


In [169]:
pirce_2024 = df[["product_code","price_eu"]]
pirce_2024["collected_timestamp"] = "13-03-2024"
pirce_2024['collected_timestamp'] = pd.to_datetime(pirce_2024['collected_timestamp'])

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_33447/2561182815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pirce_2024["collected_timestamp"] = "13-03-2024"
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_33447/2561182815.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pirce_2024['collected_timestamp'] = pd.to_datetime(pirce_2024['collected_timestamp'])
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_33447/2561182815.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [170]:
pirce_2024

,product_code,price_eu,collected_timestamp
0,4240.0,9.25,2024-03-13
1,4717.0,29.55,2024-03-13
2,4740.0,9.90,2024-03-13
3,4706.0,8.10,2024-03-13
4,4640.0,9.25,2024-03-13
...,...,...,...
1847,39643.0,0.81,2024-03-13
1848,39132.0,1.25,2024-03-13
1849,39604.0,1.25,2024-03-13
1850,39922.0,1.50,2024-03-13


In [171]:
export_to_csv(pirce_2024, "codigos-precios-2024")

DataFrame successfully exported to codigos-precios-2024.csv
